<a href="https://colab.research.google.com/github/Soniartha/Trading-Strategy/blob/main/CNNs_Jason_Brownlee_Multisteps_for_Multiple_Parallel_Input.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [37]:
#Import library
from numpy import array
from numpy import hstack
from keras.models import Sequential
from keras.layers import Flatten
from keras.layers import Dense
from keras.layers.convolutional import Conv1D
from keras.layers.convolutional import MaxPooling1D

In [16]:
#Define split function to convert sequence into X and y data
def split_sequence(sequence, n_steps_in, n_steps_out):
  X, y = list (), list ()
  for i in range(len(sequence)):
    end_ix = i + n_steps_in
    out_end_ix = end_ix + n_steps_out
    if out_end_ix > len(sequence):
      break
    seq_x, seq_y = sequence[i:end_ix, :], sequence[end_ix : out_end_ix, :]
    X.append(seq_x)
    y.append(seq_y)
  return array(X), array(y)


In [3]:
#Define sequence
in_seq1 = array([10, 20, 30, 40, 50, 60, 70, 80, 90])
in_seq2 = array([15, 25, 35, 45, 55, 65, 75, 85, 95])
out_seq = array([in_seq1[i] + in_seq2[i] for i in range (len(in_seq1))])

In [4]:
#Convert into rows and column 
in_seq1 = in_seq1.reshape((len(in_seq1), 1))
in_seq2 = in_seq2.reshape((len(in_seq2), 1))
out_seq = out_seq.reshape((len(out_seq), 1))

In [5]:
#Horizontal stack
dataset = hstack((in_seq1, in_seq2, out_seq))

In [6]:
#define n_steps_in and n_steps_out
n_steps_in, n_steps_out = 3,2

In [32]:
#Split into X and y dataset
X, y = split_sequence(dataset, n_steps_in, n_steps_out)

In [33]:
#Explore X and y 
print(X.shape, y.shape)
for i in range(len(X)):
  print(X[i],y[i])


(5, 3, 3) (5, 2, 3)
[[10 15 25]
 [20 25 45]
 [30 35 65]] [[ 40  45  85]
 [ 50  55 105]]
[[20 25 45]
 [30 35 65]
 [40 45 85]] [[ 50  55 105]
 [ 60  65 125]]
[[ 30  35  65]
 [ 40  45  85]
 [ 50  55 105]] [[ 60  65 125]
 [ 70  75 145]]
[[ 40  45  85]
 [ 50  55 105]
 [ 60  65 125]] [[ 70  75 145]
 [ 80  85 165]]
[[ 50  55 105]
 [ 60  65 125]
 [ 70  75 145]] [[ 80  85 165]
 [ 90  95 185]]


In [36]:
#Flatten output for merge y + 1 and y + 2 (2D structure)
n_output = y.shape[1] * y.shape[2]
y = y.reshape((y.shape[0], n_output))

In [38]:
#Define n_feature
n_feature = X.shape[2]

In [41]:
model = Sequential()
model.add(Conv1D(filters= 64, kernel_size= 2, activation= 'relu', input_shape = (n_steps_in, n_feature)))
model.add(MaxPooling1D(pool_size= 2))
model.add(Flatten())
model.add(Dense(50, activation= 'relu'))
model.add(Dense(n_output))
model.compile(optimizer = 'adam', loss = 'mse')

In [42]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv1d_1 (Conv1D)            (None, 2, 64)             448       
_________________________________________________________________
max_pooling1d_1 (MaxPooling1 (None, 1, 64)             0         
_________________________________________________________________
flatten_1 (Flatten)          (None, 64)                0         
_________________________________________________________________
dense_2 (Dense)              (None, 50)                3250      
_________________________________________________________________
dense_3 (Dense)              (None, 6)                 306       
Total params: 4,004
Trainable params: 4,004
Non-trainable params: 0
_________________________________________________________________


In [44]:
#Fit model
model.fit(X, y, epochs= 7000, verbose= 0)

In [48]:
#Demonstrate prediction
input_x = dataset[6:10, :]
input_x = input_x.reshape((1, n_steps_in, n_feature))
y_hat = model.predict(input_x, verbose = 0)
print(y_hat)

[[101.52633  107.70149  208.96167  112.52589  118.491135 231.39867 ]]
